# Inputs

In [1]:
splitPartCount = 5
splitSeed = 27
hoursPerWindow = 1

`functionize-notebook` has modified this notebook during execution. The following variables have been injected:

- hoursPerWindow: 8


# Preproccess

## read data

In [3]:
from utils.class_patient import Patients


patients = Patients.loadPatients()
len(patients)

1213

## remove missing

In [4]:
# fill measures whose null represent false value

from constants import NULLABLE_MEASURES


nullableMeasures = NULLABLE_MEASURES

for measureName in nullableMeasures:
    patients.fillMissingMeasureValue(measureName, 0)

In [5]:
# remove measures with less than 80% of data

measures = patients.getMeasures()

for measure, count in measures.items():
    if count < len(patients) * 80 / 100:
        patients.removeMeasures([measure])
        print(measure, count)

pco2 935
po2 935
albumin 558
hba1c 444
lymphocyte 585
height 415
urine-ketone 343
crp 57


In [6]:
# remove patients with less than 80% of data

patients.removePatientByMissingFeatures()
len(patients)

1209

## split patients

In [7]:
splitedPatients = patients.split(splitPartCount, splitSeed)

len(splitedPatients[0])

242

In [8]:
from pandas import Timedelta


splitedPatients[0].getMeasuresBetween(Timedelta(-6), Timedelta(1))

,subject_id,hadm_id,stay_id,akd,ag,age,bg,bicarbonate,bun,calcium,...,rr,saps2,sbp,scr,sodium,sofa,use_NaHCO3,uti,wbc,weight
0,13616762,20066510,31257435,True,NaN,64,NaN,NaN,NaN,NaN,...,NaN,30,NaN,NaN,NaN,3,0.0,0,NaN,130.0
0,17922352,20070680,39895564,False,NaN,47,NaN,NaN,NaN,NaN,...,NaN,26,NaN,NaN,NaN,5,0.0,0,NaN,NaN
0,11236474,20146147,34965843,True,NaN,40,NaN,NaN,NaN,NaN,...,NaN,21,NaN,NaN,NaN,3,0.0,1,NaN,NaN
0,18135181,20173681,30217596,True,NaN,54,NaN,NaN,NaN,NaN,...,NaN,24,NaN,NaN,NaN,1,0.0,0,NaN,103.5
0,12851923,20235113,35081736,False,NaN,84,NaN,NaN,NaN,NaN,...,NaN,42,NaN,NaN,NaN,5,0.0,1,NaN,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,11084536,29816115,31328473,False,NaN,25,NaN,NaN,NaN,NaN,...,NaN,17,NaN,NaN,NaN,1,0.0,0,NaN,NaN
0,11166681,29877040,38192611,True,NaN,69,NaN,NaN,NaN,NaN,...,NaN,29,NaN,NaN,NaN,4,0.0,0,NaN,NaN
0,15679298,29905973,39834726,False,NaN,50,NaN,NaN,NaN,NaN,...,NaN,32,NaN,NaN,NaN,3,0.0,0,NaN,47.4
0,18624988,29915993,31559071,True,NaN,82,NaN,NaN,NaN,NaN,...,NaN,63,NaN,NaN,NaN,11,0.0,0,NaN,120.3


In [9]:
splitedPatients = patients.split(splitPartCount, splitSeed)


def trainTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        yield trainPatients, testPatients


def trainValTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        *trainPatients, valPatients = trainPatients.split(5, 27)
        tmpPatients = Patients(patients=[])
        for trainPatientsElem in trainPatients:
            tmpPatients += trainPatientsElem
        trainPatients = tmpPatients

        yield trainPatients, valPatients, testPatients

In [10]:
for trainPatients, testPatients in trainTest():
    print(len(trainPatients.patientList), len(testPatients.patientList))

967 242
967 242
967 242
967 242
968 241


In [11]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


dataset = []
for i, (trainPatients, testPatients) in enumerate(trainTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )
    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]

    dataset.append((npTrainX, trainY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [12]:
import pickle


(TEMP_PATH / f"train-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(dataset)
)

5911442

In [13]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


datavalset = []
for i, (trainPatients, valPatients, testPatients) in enumerate(trainValTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    npValX, *_ = patientsToNumpy(
        valPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]
    valY = [p.akdPositive for p in valPatients]

    datavalset.append((npTrainX, trainY, npValX, valY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [14]:
import pickle

(TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(datavalset)
)

5708580

# LSTM

In [15]:
import pickle
from constants import TEMP_PATH

# load if not exist dataset
if "dataset" not in locals():
    dataset = pickle.loads(
        (TEMP_PATH / f"dataset-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )
   

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["AUC", "accuracy", "precision", "recall"])
    return model

2024-06-20 16:24:05.935516: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-06-20 16:24:14.339303: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npTestX, testY) in enumerate(dataset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        class_weight=weight,
        callbacks=[early_stopping],
    )

    testY = np.array(testY)
    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

2024-06-20 16:24:17.475778: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-20 16:24:17.520284: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an 

Epoch 1/50


 1/25 ━━━━━━━━━━━━━━━━━━━━ 1:03 3s/step - AUC: 0.4844 - accuracy: 0.4062 - loss: 0.6981 - precision: 0.4348 - recall: 0.6250

21/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.5701 - accuracy: 0.5260 - loss: 0.6863 - precision: 0.5212 - recall: 0.6782 

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - AUC: 0.5838 - accuracy: 0.5394 - loss: 0.6845 - precision: 0.5325 - recall: 0.6879 - val_AUC: 0.6975 - val_accuracy: 0.6753 - val_loss: 0.6642 - val_precision: 0.6809 - val_recall: 0.6598


...

23/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8968 - accuracy: 0.8177 - loss: 0.3955 - precision: 0.8514 - recall: 0.8075 

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.8994 - accuracy: 0.8212 - loss: 0.3917 - precision: 0.8526 - recall: 0.8107 - val_AUC: 0.7364 - val_accuracy: 0.6701 - val_loss: 0.7247 - val_precision: 0.6739 - val_recall: 0.6458


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - AUC: 0.5675 - accuracy: 0.4688 - loss: 0.7647 - precision: 0.5333 - recall: 0.4444

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7021 - accuracy: 0.6294 - loss: 0.6461 - precision: 0.6939 - recall: 0.5382 


Loses: [0.5651177167892456, 0.569330632686615, 0.540939211845398, 0.6286327838897705, 0.6253765821456909] 0.585879385471344 0.03496400701344809
AUCs: [0.7834938764572144, 0.7791897058486938, 0.8053563237190247, 0.7139344215393066, 0.7257198095321655] 0.761538827419281 0.03539133275577187
Accuracies: [0.6942148804664612, 0.7066115736961365, 0.7520661354064941, 0.6859503984451294, 0.6556016802787781] 0.6988889336585998 0.03146583139460219
Precisions: [0.7247706651687622, 0.7131147384643555, 0.7837837934494019, 0.6642857193946838, 0.7052631378173828] 0.7182436108589172 0.0385859301705827
Recals: [0.642276406288147, 0.707317054271698, 0.707317054271698, 0.7622950673103333, 0.5491803288459778] 0.6736771821975708 0.07294091687159952


## Static validate 

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["AUC", "accuracy", "precision", "recall"],
    )
    return model

In [19]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

if "datavalset" not in locals():
    datavalset = pickle.loads(
        (TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )

loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npValX, valY, npTestX, testY) in enumerate(datavalset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npValX = np.nan_to_num(npValX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    valY = np.array(valY)
    testY = np.array(testY)

    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_data=(npValX, valY),
        class_weight=weight,
        callbacks=[early_stopping],
    )

    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


 1/25 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - AUC: 0.4021 - accuracy: 0.3438 - loss: 0.7004 - precision: 0.4545 - recall: 0.2500

23/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.5299 - accuracy: 0.5158 - loss: 0.6895 - precision: 0.5300 - recall: 0.4933

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - AUC: 0.5401 - accuracy: 0.5238 - loss: 0.6881 - precision: 0.5378 - recall: 0.5006 - val_AUC: 0.7584 - val_accuracy: 0.7202 - val_loss: 0.6455 - val_precision: 0.7157 - val_recall: 0.7449


Epoch 2/50


...

22/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.9107 - accuracy: 0.8441 - loss: 0.3658 - precision: 0.8480 - recall: 0.7904 

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.9102 - accuracy: 0.8428 - loss: 0.3669 - precision: 0.8520 - recall: 0.7902 - val_AUC: 0.7413 - val_accuracy: 0.6580 - val_loss: 0.6967 - val_precision: 0.6739 - val_recall: 0.6327


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - AUC: 0.5714 - accuracy: 0.5312 - loss: 0.7407 - precision: 0.5882 - recall: 0.5556

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7122 - accuracy: 0.6469 - loss: 0.6271 - precision: 0.6760 - recall: 0.6374 


Loses: [0.5743482708930969, 0.5753347277641296, 0.5340223908424377, 0.619045078754425, 0.6119911670684814] 0.5829483270645142 0.030568405695175414
AUCs: [0.7721527814865112, 0.7714353799819946, 0.8111635446548462, 0.731010913848877, 0.7319533824920654] 0.7635432004928588 0.02986762406248804
Accuracies: [0.6735537052154541, 0.71074378490448, 0.7644628286361694, 0.6776859760284424, 0.6597510576248169] 0.6972394704818725 0.03755018537824486
Precisions: [0.699999988079071, 0.7086614370346069, 0.7844827771186829, 0.6641790866851807, 0.6724137663841248] 0.7059474110603332 0.042613706117643
Recals: [0.6260162591934204, 0.7317073345184326, 0.7398374080657959, 0.7295082211494446, 0.6393442749977112] 0.6932826995849609 0.0497799347408734
